In [5]:
!pip install autogluon --upgrade
!pip install torch torchvision torchaudio --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 703.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 12.2 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.3.1
    Uninstalling triton-2.3.1:
      Successfully uninstalled triton-2.3.1
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 8.9.2.26
    Uninstalling nvidia-cudnn-cu12-8.9.2.26:
      Successfully uninstalled nvidia-cudnn-cu12-8.9.2.26
  Attempting uninstall: torch
    Found existing installation: torch 2.3.1
    Uninstalling torch-2.3.1:
      Successfully uninstalled torch-2.3.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.18.1
    Uninsta

In [1]:
!wget --quiet https://automl-mm-bench.s3.amazonaws.com/multilingual-datasets/amazon_review_sentiment_cross_lingual.zip -O amazon_review_sentiment_cross_lingual.zip
!unzip -q -o amazon_review_sentiment_cross_lingual.zip -d .

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

train_de_df = pd.read_csv('amazon_review_sentiment_cross_lingual/de_train.tsv',
                          sep='\t', header=None, names=['label', 'text']) \
                .sample(1000, random_state=123)
train_de_df.reset_index(inplace=True, drop=True)

test_de_df = pd.read_csv('amazon_review_sentiment_cross_lingual/de_test.tsv',
                          sep='\t', header=None, names=['label', 'text']) \
               .sample(200, random_state=123)
test_de_df.reset_index(inplace=True, drop=True)
print(train_de_df)

     label                                               text
0        0  Dieser Film, nur so triefend von Kitsch, ist h...
1        0  Wie so oft: Das Buch begeistert, der Film entt...
2        1  Schon immer versuchten Männer ihre Gefühle geg...
3        1  Wenn man sich durch 10 Minuten Disney-Trailer ...
4        1  Eine echt geile nummer zum Abtanzen und feiern...
..     ...                                                ...
995      0  Ich dachte dies wäre ein richtig spannendes Bu...
996      0  Wer sich den Schrott wirklich noch ansehen möc...
997      0  Sicher, der Film greift ein aktuelles und hoch...
998      1  Dieser Bildband lässt das Herz von Sarah Kay-F...
999      1  ...so das war nun mein drittes Buch von Jenny-...

[1000 rows x 2 columns]


In [3]:
train_en_df = pd.read_csv('amazon_review_sentiment_cross_lingual/en_train.tsv',
                          sep='\t',
                          header=None,
                          names=['label', 'text']) \
                .sample(1000, random_state=123)
train_en_df.reset_index(inplace=True, drop=True)

test_en_df = pd.read_csv('amazon_review_sentiment_cross_lingual/en_test.tsv',
                          sep='\t',
                          header=None,
                          names=['label', 'text']) \
               .sample(200, random_state=123)
test_en_df.reset_index(inplace=True, drop=True)
print(train_en_df)

     label                                               text
0        0  This is a film that literally sees little wron...
1        0  This music is pretty intelligent, but not very...
2        0  One of the best pieces of rock ever recorded, ...
3        0  Reading the posted reviews here, is like revis...
4        1  I've just finished page 341, the last page. It...
..     ...                                                ...
995      1  This album deserves to be (at least) as popula...
996      1  This book, one of the few that takes a more ac...
997      1  I loved it because it really did show Sagan th...
998      1  Stuart Gordons "DAGON" is a unique horror gem ...
999      0  I've heard Al Lee speak before and thought tha...

[1000 rows x 2 columns]


In [4]:
from autogluon.multimodal import MultiModalPredictor

predictor = MultiModalPredictor(label='label')
predictor.fit(train_de_df,
              hyperparameters={
                  'model.hf_text.checkpoint_name': 'bert-base-multilingual-cased',
                  'optimization.max_epochs': 2
              },
               time_limit=300)

No path specified. Models will be saved in: "AutogluonModels/ag-20240914_085627"
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Pytorch Version:    2.4.1+cu121
CUDA Version:       CUDA is not available
Memory Avail:       10.56 GB / 12.67 GB (83.3%)
Disk Space Avail:   62.24 GB / 107.72 GB (57.8%)
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Time limit reached. Elapsed time is 0:05:22. Signaling Trainer to stop.


Validation: |          | 0/? [00:00<?, ?it/s]

AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/content/AutogluonModels/ag-20240914_085627")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).




In [5]:
score = predictor.evaluate(test_de_df)
print('Score on the German Testset:')
print(score)

Predicting: |          | 0/? [00:00<?, ?it/s]

Score on the German Testset:
{'roc_auc': 0.5173277243589743}


In [6]:
score = predictor.evaluate(test_en_df)
print('Score on the English Testset:')
print(score)

Predicting: |          | 0/? [00:00<?, ?it/s]

Score on the English Testset:
{'roc_auc': 0.49241282283187615}


In [7]:
!zip -r /content/AutogluonModels.zip /content/AutogluonModels

  adding: content/AutogluonModels/ (stored 0%)
  adding: content/AutogluonModels/ag-20240914_082944/ (stored 0%)
  adding: content/AutogluonModels/ag-20240914_085627/ (stored 0%)
  adding: content/AutogluonModels/ag-20240914_085627/hparams.yaml (deflated 36%)
  adding: content/AutogluonModels/ag-20240914_085627/config.yaml (deflated 58%)
  adding: content/AutogluonModels/ag-20240914_085627/data_processors.pkl (deflated 77%)
  adding: content/AutogluonModels/ag-20240914_085627/events.out.tfevents.1726304193.f3aef5ba414d.10098.0 (deflated 38%)
  adding: content/AutogluonModels/ag-20240914_085627/assets.json (deflated 44%)
  adding: content/AutogluonModels/ag-20240914_085627/hf_text/ (stored 0%)
  adding: content/AutogluonModels/ag-20240914_085627/hf_text/special_tokens_map.json (deflated 42%)
  adding: content/AutogluonModels/ag-20240914_085627/hf_text/vocab.txt (deflated 45%)
  adding: content/AutogluonModels/ag-20240914_085627/hf_text/tokenizer_config.json (deflated 76%)
  adding: cont